## Import

In [ ]:
#General
import pandas as pd
import numpy as np
from numpy import std, mean, sqrt
import math
#Statistics
from scipy.stats import mannwhitneyu
#Visualization
from matplotlib import pyplot as plt
import matplotlib
#Utility
from tabulate import tabulate

In [ ]:
#Custom imports
from stat_util import normal_test_all_model_samples
from stat_util import t_test_model_samples_against_other_model_samples
from stat_util import cohen_d_test_model_samples_against_other_model_samples

## Dataset preparation

In [ ]:
source_datasets = [
    "JDT_R2_0","JDT_R2_1","JDT_R3_0","JDT_R3_1","JDT_R3_2"
]
target_datasets = [
    "PDE_R2_0","PDE_R2_1","PDE_R3_0","PDE_R3_1","PDE_R3_2"    
]

In [ ]:
main_model_name = "REPDX"

In [ ]:
main_df = None

for source in source_datasets:
    for target in target_datasets:
        print("Analyzing: S ", source, " T ", target)
        results_df = pd.read_csv("results_v2/cross_balance_"+source+"_"+target)
        #results_df = pd.read_csv("results_v2/cross_hp_"+source+"_"+target)
        #
        results_df.loc[results_df["Model"]=="OLD_REPD", 'Model'] = "REPD"
        results_df.loc[results_df["Model"]=="REPD_EX", 'Model'] = "REPDX"
        #
        #results_df = results_df[results_df["Model"]!="REPD"]
        #
        if main_df is None:
            main_df = results_df
        else: 
            main_df = pd.concat([main_df,results_df])

        #
        #print(results_df[results_df["F1 score"]==results_df["F1 score"].max()][["Model","F1 score","Precision","Recall"]])
        #print(results_df[results_df["Precision"]==results_df["Precision"].max()][["Model","F1 score","Precision","Recall"]])
        #print()
        #
        #group_df = results_df \
        #            .drop(columns=['Accuracy']) \
        #            .rename(index=str, columns={"F1 score": "F1"}) \
        #            .groupby(["Source","Target","Model"])
        #print(group_df["F1"].median())
        #print()
        #
        #
        stat, p = mannwhitneyu(
                    results_df[results_df["Model"]=="REPDX"]["F1 score"].values, 
                    results_df[results_df["Model"]=="TCA+"]["F1 score"].values
                    #results_df[results_df["Model"]=="REPD"]["F1 score"].values
                    )
        #print('Statistics=%.3f, p=%.3f' % (stat, p))
        alpha = 0.05
        if p > alpha:
            #print('Same distribution (fail to reject H0)')
            print('NO')
        else:
            #print('Different distribution (reject H0)')
            print('YES')
        print()
        #print()

In [ ]:
group_df = main_df.drop(columns=['Accuracy']).rename(index=str, columns={"F1 score": "F1"}).groupby(["Source", "Target", "Model"])

In [ ]:
#print(group_df[["F1","Precision","Recall"]].median().to_latex())
#print(group_df[["Precision"]].median().to_latex())
#print(group_df[["Recall","Precision"]].median().to_latex())
#
print(group_df[["F1","Precision","Recall"]].median())


In [ ]:
import seaborn as sns

In [ ]:
def plot_measure(measure_name,plot_name):
    df = main_df.drop(columns=['Accuracy'])
    new_dfs = []
    
    for source in source_datasets:
        for target in target_datasets:
            new_df = []
            for model in ["REPD","REPDX","TCA+"]:#list(set(df.Model.values))
                f1_scores = []
                temp = df.loc[(df['Source'] == source) & (df['Target'] == target) & (df['Model'] == model)][measure_name].values
                for value in temp:
                    d = {
                        'DatasetModel' : '{: <30}'.format(", ".join([source, target, model])), measure_name : value
                    }
                    new_df.append(d)

            new_df = pd.DataFrame(new_df)
            new_dfs.append(new_df)
    #Plotting prepared dataset
    fig, axs = plt.subplots(25, 1, figsize=(64,128), sharex=True, sharey="row", gridspec_kw={'hspace': 0.1})

    font = {'size': 60}
    matplotlib.rc('font', **font)
    for n, new_df in enumerate(new_dfs):
        ax=axs[n]
        sns.violinplot(measure_name,'DatasetModel', data=new_df, ax = ax, scale="count", cut=0)
        #
        yax = ax.get_yaxis()
        yax.grid(True)
        yax.set_ticks_position('right')
        pad = max([len(i) for i in new_df['DatasetModel'].values])
        yax.set_tick_params(pad=pad)
        y_label = yax.get_label()
        y_label.set_visible(False)
        #
        xax = ax.get_xaxis()
        xax.grid(True)
        x_label = xax.get_label()
        x_label.set_visible(False)
        #
        ax.set_title("")

    fig.suptitle(measure_name)
    st = fig.suptitle(measure_name, fontsize="x-large")
    st.set_y(0.95)
    fig.subplots_adjust(top=0.93)
    fig.tight_layout()
    plt.tight_layout()
    plt.savefig(plot_name,format='pdf',bbox_inches='tight')
    plt.show()

In [ ]:
plot_measure("F1 score","cross-f1.pdf")

In [ ]:
plot_measure("Precision","cross-precision.pdf")

In [ ]:
plot_measure("Recall","cross-recall.pdf")